In [1]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import tensorflow as tf
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import copy
import glob
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import synonyms
from opencc import OpenCC

t2s_cc = OpenCC('t2s')
s2t_cc = OpenCC('s2tw')

Init Plugin
Init Graph Optimizer
Init Kernel
smart_open library not found; falling back to local-filesystem-only
[jieba] default dict file path ../data/vocab.txt
[jieba] default dict file path ../data/vocab.txt
[jieba] load default dict ../data/vocab.txt ...
[jieba] load default dict ../data/vocab.txt ...
>> Synonyms load wordseg dict [/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/synonyms/data/vocab.txt] ... 
>> Synonyms on loading stopwords [/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/synonyms/data/stopwords.txt] ...
[Synonyms] on loading vectors [/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/synonyms/data/words.vector.gz] ...


In [2]:
# data_utils.download_data_gdown("./") # gdrive-ckip

In [3]:
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Metal device set to: Apple M1 Pro


2022-04-19 21:13:56.779067: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-19 21:13:56.779168: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-04-19 21:13:56.858119: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-19 21:13:56.862846: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-04-19 21:13:57.170588: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-04-19 21:13:59.999503: I tensorflow/core/common_run

In [4]:
tsv_list = glob.glob("./train/*.tsv")
csv_list = glob.glob("./train/*.csv")

In [5]:
csv_list = tsv_list + csv_list

In [6]:
print(os.path.splitext(csv_list[0])[0])

./train/train


In [7]:
print(os.path.splitext(csv_list[0])[0].rsplit('/', 2)[2])

train


In [8]:
with open('keyword.txt', encoding='utf-8') as f:
    offensive_keyword_list = f.read().replace('\n', ' ').split()
    
dict_for_CKIP = dict((el,2) for el in offensive_keyword_list)

dict_for_CKIP = construct_dictionary(dict_for_CKIP)

In [9]:
# 讀取 stop-words.txt
with open('stopword.txt', encoding='utf-8') as f:
    stop_word = f.read().replace('\n', ' ').split()

# 整理 stop-word
sw = stop_word

In [10]:
augmented_df = pd.DataFrame(columns=['text', 'label'])

In [11]:
for csv in csv_list:
    filename = os.path.splitext(csv)[0].rsplit('/', 2)[2]
    
    df = pd.read_csv(csv, sep="\t") #'\t'
    sentence_list = df['text'].tolist()
    label_list = df['label'].tolist()
    word_sentence_list = ws(
    sentence_list,
        # sentence_segmentation = True, # To consider delimiters
        # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
        recommend_dictionary = dict_for_CKIP, # words in this dictionary are encouraged
        # coerce_dictionary = dictionary2, # words in this dictionary are forced
    )

    pos_sentence_list = pos(word_sentence_list)
    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
    
    df_result = pd.DataFrame(sentence_list, columns=['text'])
    df_result['WS'] = np.array(word_sentence_list)
    df_result['POS'] = np.array(pos_sentence_list)
    df_result['NER'] = np.array(entity_sentence_list)
    df_result.to_csv("./ws/ws_"+filename+".csv")
    
    word_segment = df_result['WS'].tolist()
    cut_corpus = []
    for i in word_segment:
        cut_corpus.append(' '.join(i))

    for c in cut_corpus:
        print (c)
    
    text_cv = CountVectorizer(max_df=0.75, min_df=1, stop_words=sw)

    td_matrix = text_cv.fit_transform(cut_corpus)

    print (td_matrix.shape)

    print (text_cv.vocabulary_.keys())

    print (len(text_cv.vocabulary_.keys()))

    print(text_cv.get_feature_names_out())

    print('============全部字詞=============')
    print(text_cv.get_feature_names())
    print()
    print('==========各文件字詞統計==============')
    print(td_matrix.toarray())
    print()
    
    tfidf = TfidfTransformer()

    tfidf_matrix = tfidf.fit_transform(td_matrix)

    print('==============IDF============')
    print(tfidf.idf_)
    print()
 
    df = pd.DataFrame(tfidf_matrix.T.toarray(), index=text_cv.get_feature_names())
    print('============TFIDF============')
    print(df)
    print()
    
    idf_df = pd.DataFrame(tfidf_matrix.T.toarray(), index=text_cv.vocabulary_.keys())
    
    idf_df.to_csv("./idf/idf_"+filename+".csv")
    
    # Step4: 關鍵字萃取(印出各文件tfidf數值top 10)
    print('============關鍵字萃取============')
    for i, vector in enumerate(tfidf_matrix.toarray()):
        doc_tfidf = list(zip(text_cv.get_feature_names_out(), vector))
        doc_tfidf = sorted(doc_tfidf, key=lambda x: x[1], reverse=True)
        top_keywords = [x[0] for x in doc_tfidf[:] if x[1] != 0.0]
        print(sentence_list[i])
        print('doc' + str(i+1) + ": " + '、'.join(top_keywords))
        for keyword in top_keywords:
            if keyword in sentence_list[i]:
                synonyms_tuple = synonyms.nearby(t2s_cc.convert(keyword))
                for x in range(0, 10):
                    if len(synonyms_tuple[1]) > 0 and synonyms_tuple[1][x] > 0.5:
                        new_sentence = sentence_list[i].replace(keyword, s2t_cc.convert(synonyms_tuple[0][x]))
                        new_df = pd.DataFrame([[new_sentence, label_list[i]]], columns=['text', 'label'])
                        augmented_df = pd.concat([augmented_df, new_df])
    
    augmented_df.to_csv("./augmented/augmented_"+filename+".csv", sep="\t",index=False)

2022-04-19 21:14:03.802184: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-04-19 21:14:08.666260: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-04-19 21:14:15.274290: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
/var/folders/rd/yxk_41h92vz6r66k9g09_fjw0000gn/T/ipykernel_2352/1445789412.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df_result['WS'] = np.array(word_sentence_list)
/var/folders/rd/yxk_41h92vz6r66k9g09_fjw0000gn/T/ipykernel_2352/1445789412.py:20: VisibleDeprecationWarning: Creating an ndarray from ragg

欠告 ， 不 支持 這 種 恐嚇 行為
憨粉 就 是 欠 吉
那些 欠缺 新聞 素養 的 媒體 內容 就 是 這 年頭 臺灣 的 亂源
雖然 罵 韓 沒用 但 他 欠罵 也 是 事實
真的 很 欠打
臺灣人 欠 阿扁 一 個 公道
真的 是 欠罵
怎麼 會 有 怎麼 欠揍 的 狗 哈哈 哈哈
笑死哈哈 哈哈 哈欠 超渡 林筱恩
賴洛琦 這 隻 真的 很 欠揍 耶
這 是 欠 我 錢 的
破麻欠操 ， 開飯 才 是 王道
欠揍哈哈
那些 大人 是 不 是 也 欠 教育
可悲 欠告 沒 告 都 不會 怕
有夠 欠揍 的
韓國瑜 你 的 捐款 收據 也 欠 很 久 了 耶
後 你 真的 很 欠揍 耶
臺灣 不 缺 媒體 中國時報 滾出去 我 很 理性 又 冷靜 唷
幹 中 夭 電視 滾
東森 廢文 滾出 臺灣
洋 垃圾 滾出 中國 ， 不 歡迎 你 這 種 不 懂 裝懂
再 講 不 聽 就 給 我 滾出去 不要 再 進來 我們 班
通通 給 我 滾回去
給 我 眼睛 閉起來 睡覺 喔 要不然 就 給 我 滾出去 有沒有 聽到
國民黨 滾出 臺灣 啦 臺灣 青年 群 超想 幹爆 你們 的
臺灣 二號 臺灣 噩耗 快 滾 啦 韓國 輸
滾回 妳 家 廁所 吃飯
中國 網軍 五毛 滾開 喔 慢 走 不 送
付不起 錢 就 滾蛋麥 靠 ㄧㄠ
滾去 中國 好好 賺錢 啦 丟臉
你 少 貼 了 支 那 賤 畜 滾回 中國 的 照片 吧 如果 你 是 中立 的
滾去 大陸 吧 韓粉 們
一 個 人 有 人 喜歡 你 有 人 討厭 你 很 正常 ， 霸氣 的 說 不 喜歡 滾
滾岀 地球 好 嗎 不男不女 的
陳宏 去 吃屎 看吃 得 飽 嗎
公視 水準 很 高 啊 可是 大家 就 愛 吃屎
館長 很 男人 嗎 感覺 像 頭 豬 說話 沒有 經過 大腦 怪不得 和 吃 屎哥 也 能 切磋
支持 呱吉 ， 義和團 去 吃屎
療癒 ， 選擇性 無視 只 打 臺派 、 不 禮貌 根本 就 應該 去 吃屎
吃屎 黨 黨主席
沒辦法 的 ， 官 都 是 吃屎 長大 的
這 是 館長 對 吃 屎哥 吧
動漫 還是 日劇 聽 過 く そ っ た れ ， 好像 是 吃屎 吧 的 意思
欠扁 ， 來人 餵 公子 吃屎
只 想 叫 他 去 吃屎
叫 他 吃屎 順便 踹 他 一 腳 叫 他 閉上 狗嘴 ， 因

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'lex', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1

============關鍵字萃取============
欠告，不支持這種恐嚇行為
doc1: 恐嚇、欠告、支持
憨粉就是欠吉
doc2: 憨粉
那些欠缺新聞素養的媒體內容就是這年頭臺灣的亂源
doc3: 亂源、年頭、欠缺、素養、內容、新聞、媒體、臺灣
雖然罵韓沒用但他欠罵也是事實
doc4: 事實、欠罵、沒用
真的很欠打
doc5: 欠打、真的
臺灣人欠阿扁一個公道
doc6: 公道、阿扁、臺灣人
真的是欠罵
doc7: 欠罵、真的
怎麼會有怎麼欠揍的狗哈哈哈哈
doc8: 欠揍
笑死哈哈哈哈哈欠超渡林筱恩
doc9: 哈欠、林筱恩、笑死哈哈、超渡
賴洛琦這隻真的很欠揍耶
doc10: 賴洛琦、欠揍、真的
這是欠我錢的
doc11: 
破麻欠操，開飯才是王道
doc12: 破麻欠操、開飯、王道
欠揍哈哈
doc13: 欠揍哈哈
那些大人是不是也欠教育
doc14: 大人、教育
可悲欠告沒告都不會怕
doc15: 可悲、欠告
有夠欠揍的
doc16: 欠揍、有夠
韓國瑜你的捐款收據也欠很久了耶
doc17: 捐款、收據、韓國瑜
後你真的很欠揍耶
doc18: 欠揍、真的
臺灣不缺媒體中國時報滾出去我很理性又冷靜唷
doc19: 中國時報、冷靜、理性、滾出去、媒體、臺灣
幹中夭電視滾
doc20: 電視
東森廢文滾出臺灣
doc21: 東森、廢文、滾出、臺灣
洋垃圾滾出中國，不歡迎你這種不懂裝懂
doc22: 裝懂、滾出、中國、垃圾
再講不聽就給我滾出去不要再進來我們班
doc23: 滾出去
通通給我滾回去
doc24: 滾回去、通通
給我眼睛閉起來睡覺喔要不然就給我滾出去有沒有聽到
doc25: 有沒有、睡覺、聽到、閉起來、眼睛、滾出去
國民黨滾出臺灣啦臺灣青年群超想幹爆你們的
doc26: 臺灣、幹爆、超想、青年、滾出、國民黨
臺灣二號臺灣噩耗快滾啦韓國輸
doc27: 臺灣、二號、噩耗、韓國
滾回妳家廁所吃飯
doc28: 吃飯、滾回、廁所
中國網軍五毛滾開喔慢走不送
doc29: 五毛、滾開、網軍、中國
付不起錢就滾蛋麥靠ㄧㄠ
doc30: ㄧㄠ、付不起、滾蛋麥
滾去中國好好賺錢啦丟臉
doc31: 賺錢、丟臉、滾去、中國、好好
你少貼了支那賤畜滾回中國的照片吧如果你是中立的
doc32: 照片、中立、滾回、中國
滾去大陸吧韓粉們
doc33: 大陸、滾去、

智齒本來就是智障之齒
doc255: 本來、智齒、智障
所以你是智障還是智齒是智障
doc256: 智齒、智障
沒智齒前覺得自己是智障有了智齒才知道不是牙齒問題
doc257: 智齒、牙齒、智障
智障的牙齒啊
doc258: 牙齒、智障
就是智障牙齒*｀*
doc259: 牙齒、智障
但是智障的智代表智能啊
doc260: 代表、智能、智障
臺灣智障客人多，看店家譙智障蠻愉快的
doc261: 店家、愉快、譙智障、客人、臺灣、智障
罵高學歷智障太過分吧這是歧視嗎那這樣你才智障你全家都智障欸
doc262: 智障、過分、歧視、高學歷
低學歷智障簽
doc263: 低學歷、智障
習近平也是最高領導人最高智障者
doc264: 智障者、習近平、領導人
香港有個年年考第一的智障
doc265: 年年、第一、香港、智障
講得好像低學歷智障很好用一樣智障不分學歷
doc266: 好用、低學歷、學歷、智障
高學歷低智障，好多香港醫生律師都是這樣
doc267: 好多、律師、醫生、香港、高學歷、智障
老闆智障，用任何人才都難用
doc268: 人才、老闆、智障
唉智不智障跟學歷沒有直接關聯
doc269: 關聯、學歷、智障
高學歷智障，就是因為整天只會讀書，沒有實戰經驗
doc270: 實戰、經驗、讀書、高學歷、智障
高知識文盲也很多啊，比高學歷智障更可怕
doc271: 文盲、知識、可怕、高學歷、智障
也不一定吧低學歷也一樣有智障
doc272: 不一定、學歷、智障
因為一直有智障在傳這個癈文謠言
doc273: 癈文、謠言、智障
高學歷固然有智障，可是低學歷的智障一定更多
doc274: 更多、低學歷、智障、高學歷
高學歷智障你指臺大法律系的政治官員嗎
doc275: 法律系、臺大、官員、政治、高學歷、智障
風傳媒小編屬於哪一類高學歷智障還是低學歷天才
doc276: 天才、傳媒、低學歷、小編、高學歷、智障
出社會不會很難用阿，你看看我們政府裡頭多少高學歷智障
doc277: 出社、裡頭、會不會、政府、高學歷、智障
先不說是不是智障了請問小編的學歷是什麼
doc278: 請問、學歷、小編、智障
非智障，只是不懂得合群的重要性
doc279: 合群、重要性、懂得、智障
可不可以請風傳媒小編，不要用智障這個字來罵人
doc280: 可不可以、傳媒、罵人、小編、智障
現在也是一堆智障

...

欠告 ， 不 支持 這 種 恐嚇 行為
憨粉 就 是 欠 吉
那些 欠缺 新聞 素養 的 媒體 內容 就 是 這 年頭 臺灣 的 亂源
雖然 罵 韓 沒用 但 他 欠罵 也 是 事實
真的 很 欠打
臺灣人 欠 阿扁 一 個 公道
真的 是 欠罵
怎麼 會 有 怎麼 欠揍 的 狗 哈哈 哈哈
笑死哈哈 哈哈 哈欠 超渡 林筱恩
賴洛琦 這 隻 真的 很 欠揍 耶
這 是 欠 我 錢 的
破麻欠操 ， 開飯 才 是 王道
欠揍哈哈
那些 大人 是 不 是 也 欠 教育
可悲 欠告 沒 告 都 不會 怕
有夠 欠揍 的
韓國瑜 你 的 捐款 收據 也 欠 很 久 了 耶
後 你 真的 很 欠揍 耶
臺灣 不 缺 媒體 中國時報 滾出去 我 很 理性 又 冷靜 唷
幹 中 夭 電視 滾
東森 廢文 滾出 臺灣
洋 垃圾 滾出 中國 ， 不 歡迎 你 這 種 不 懂 裝懂
再 講 不 聽 就 給 我 滾出去 不要 再 進來 我們 班
通通 給 我 滾回去
給 我 眼睛 閉起來 睡覺 喔 要不然 就 給 我 滾出去 有沒有 聽到
國民黨 滾出 臺灣 啦 臺灣 青年 群 超想 幹爆 你們 的
臺灣 二號 臺灣 噩耗 快 滾 啦 韓國 輸
滾回 妳 家 廁所 吃飯
中國 網軍 五毛 滾開 喔 慢 走 不 送
付不起 錢 就 滾蛋麥 靠 ㄧㄠ
滾去 中國 好好 賺錢 啦 丟臉
你 少 貼 了 支 那 賤 畜 滾回 中國 的 照片 吧 如果 你 是 中立 的
滾去 大陸 吧 韓粉 們
一 個 人 有 人 喜歡 你 有 人 討厭 你 很 正常 ， 霸氣 的 說 不 喜歡 滾
滾岀 地球 好 嗎 不男不女 的
陳宏 去 吃屎 看吃 得 飽 嗎
公視 水準 很 高 啊 可是 大家 就 愛 吃屎
館長 很 男人 嗎 感覺 像 頭 豬 說話 沒有 經過 大腦 怪不得 和 吃 屎哥 也 能 切磋
支持 呱吉 ， 義和團 去 吃屎
療癒 ， 選擇性 無視 只 打 臺派 、 不 禮貌 根本 就 應該 去 吃屎
吃屎 黨 黨主席
沒辦法 的 ， 官 都 是 吃屎 長大 的
這 是 館長 對 吃 屎哥 吧
動漫 還是 日劇 聽 過 く そ っ た れ ， 好像 是 吃屎 吧 的 意思
欠扁 ， 來人 餵 公子 吃屎
只 想 叫 他 去 吃屎
叫 他 吃屎 順便 踹 他 一 腳 叫 他 閉上 狗嘴 ， 因

============關鍵字萃取============
欠告，不支持這種恐嚇行為
doc1: 恐嚇、欠告、支持
憨粉就是欠吉
doc2: 憨粉
那些欠缺新聞素養的媒體內容就是這年頭臺灣的亂源
doc3: 亂源、年頭、欠缺、素養、內容、新聞、媒體、臺灣
雖然罵韓沒用但他欠罵也是事實
doc4: 事實、欠罵、沒用
真的很欠打
doc5: 欠打、真的
臺灣人欠阿扁一個公道
doc6: 公道、阿扁、臺灣人
真的是欠罵
doc7: 欠罵、真的
怎麼會有怎麼欠揍的狗哈哈哈哈
doc8: 欠揍
笑死哈哈哈哈哈欠超渡林筱恩
doc9: 哈欠、林筱恩、笑死哈哈、超渡
賴洛琦這隻真的很欠揍耶
doc10: 賴洛琦、欠揍、真的
這是欠我錢的
doc11: 
破麻欠操，開飯才是王道
doc12: 破麻欠操、開飯、王道
欠揍哈哈
doc13: 欠揍哈哈
那些大人是不是也欠教育
doc14: 大人、教育
可悲欠告沒告都不會怕
doc15: 可悲、欠告
有夠欠揍的
doc16: 欠揍、有夠
韓國瑜你的捐款收據也欠很久了耶
doc17: 捐款、收據、韓國瑜
後你真的很欠揍耶
doc18: 欠揍、真的
臺灣不缺媒體中國時報滾出去我很理性又冷靜唷
doc19: 中國時報、冷靜、理性、滾出去、媒體、臺灣
幹中夭電視滾
doc20: 電視
東森廢文滾出臺灣
doc21: 東森、廢文、滾出、臺灣
洋垃圾滾出中國，不歡迎你這種不懂裝懂
doc22: 裝懂、滾出、中國、垃圾
再講不聽就給我滾出去不要再進來我們班
doc23: 滾出去
通通給我滾回去
doc24: 滾回去、通通
給我眼睛閉起來睡覺喔要不然就給我滾出去有沒有聽到
doc25: 有沒有、睡覺、聽到、閉起來、眼睛、滾出去
國民黨滾出臺灣啦臺灣青年群超想幹爆你們的
doc26: 臺灣、幹爆、超想、青年、滾出、國民黨
臺灣二號臺灣噩耗快滾啦韓國輸
doc27: 臺灣、二號、噩耗、韓國
滾回妳家廁所吃飯
doc28: 吃飯、滾回、廁所
中國網軍五毛滾開喔慢走不送
doc29: 五毛、滾開、網軍、中國
付不起錢就滾蛋麥靠ㄧㄠ
doc30: ㄧㄠ、付不起、滾蛋麥
滾去中國好好賺錢啦丟臉
doc31: 賺錢、丟臉、滾去、中國、好好
你少貼了支那賤畜滾回中國的照片吧如果你是中立的
doc32: 照片、中立、滾回、中國
滾去大陸吧韓粉們
doc33: 大陸、滾去、

智障的牙齒簡稱智齒
doc254: 簡稱、智齒、牙齒、智障
智齒本來就是智障之齒
doc255: 本來、智齒、智障
所以你是智障還是智齒是智障
doc256: 智齒、智障
沒智齒前覺得自己是智障有了智齒才知道不是牙齒問題
doc257: 智齒、牙齒、智障
智障的牙齒啊
doc258: 牙齒、智障
就是智障牙齒*｀*
doc259: 牙齒、智障
但是智障的智代表智能啊
doc260: 代表、智能、智障
臺灣智障客人多，看店家譙智障蠻愉快的
doc261: 店家、愉快、譙智障、客人、臺灣、智障
罵高學歷智障太過分吧這是歧視嗎那這樣你才智障你全家都智障欸
doc262: 智障、過分、歧視、高學歷
低學歷智障簽
doc263: 低學歷、智障
習近平也是最高領導人最高智障者
doc264: 智障者、習近平、領導人
香港有個年年考第一的智障
doc265: 年年、第一、香港、智障
講得好像低學歷智障很好用一樣智障不分學歷
doc266: 好用、低學歷、學歷、智障
高學歷低智障，好多香港醫生律師都是這樣
doc267: 好多、律師、醫生、香港、高學歷、智障
老闆智障，用任何人才都難用
doc268: 人才、老闆、智障
唉智不智障跟學歷沒有直接關聯
doc269: 關聯、學歷、智障
高學歷智障，就是因為整天只會讀書，沒有實戰經驗
doc270: 實戰、經驗、讀書、高學歷、智障
高知識文盲也很多啊，比高學歷智障更可怕
doc271: 文盲、知識、可怕、高學歷、智障
也不一定吧低學歷也一樣有智障
doc272: 不一定、學歷、智障
因為一直有智障在傳這個癈文謠言
doc273: 癈文、謠言、智障
高學歷固然有智障，可是低學歷的智障一定更多
doc274: 更多、低學歷、智障、高學歷
高學歷智障你指臺大法律系的政治官員嗎
doc275: 法律系、臺大、官員、政治、高學歷、智障
風傳媒小編屬於哪一類高學歷智障還是低學歷天才
doc276: 天才、傳媒、低學歷、小編、高學歷、智障
出社會不會很難用阿，你看看我們政府裡頭多少高學歷智障
doc277: 出社、裡頭、會不會、政府、高學歷、智障
先不說是不是智障了請問小編的學歷是什麼
doc278: 請問、學歷、小編、智障
非智障，只是不懂得合群的重要性
doc279: 合群、重要性、懂得、智障
可不可以請風傳媒小編，不要用智障這個字來罵人
doc

真的是幹你娘看到這種的真的是氣到很無奈
doc496: 真的、無奈
我做做效果你也在爽阿屁孩
doc497: 做做、效果、阿屁孩
看到這些小屁孩跟恐龍家長，我突然覺得我小孩的時候怎麼這麼乖巧
doc498: 乖巧、家長、恐龍、小孩、屁孩
你老婆為你好才檢舉你，不要講話跟屁孩一樣
doc499: 講話、檢舉、老婆、屁孩
賴巧瑩小屁孩
doc500: 賴巧瑩、小屁孩
太多教授都是自以為是大老闆的老屁孩
doc501: 教授、自以為是、太多、老闆、老屁孩
超級大屁孩
doc502: 超級、屁孩
立馬連署關掉頻道死屁孩
doc503: 連署、關掉、頻道、死屁孩
就是白目死屁孩
doc504: 白目、死屁孩
弱智與屁孩不分國界
doc505: 國界、弱智、屁孩
根本是屁孩一個
doc506: 屁孩
話說他才歲確實是個屁孩
doc507: 才歲、確實、屁孩
就是個屁孩
doc508: 屁孩
直接戳破我不能接受小屁孩ㄉ弱智行為同是畫手真的不能忍
doc509: 戳破、接受、畫手、小屁孩、弱智、真的
寵物鼠是無辜的，該死的只有帶他進考場的無腦屁孩
doc510: 寵物鼠、無辜、腦屁、考場、該死
帶寵物去考場影響到其他考生然後那小屁孩居然還能繼續考
doc511: 寵物、影響到、考生、考場、小屁孩
屁孩臉書又開了胡寧耀
doc512: 胡寧耀、臉書、屁孩
屁孩沒死欸好可惜
doc513: 可惜、屁孩
屁孩騎這樣是在騎三小車幹你怎不是遇到砂石車
doc514: 怎不是、砂石車、屁孩
沒出事就車友車友的叫，出事了就淦林良屁孩
doc515: 出事、車友、林良、屁孩
屁孩沒掛
doc516: 屁孩
這種屁孩給他死趕快去死一死
doc517: 屁孩
斃了這屁孩吧
doc518: 屁孩
走到哪就是有這種腦殘屁孩
doc519: 腦殘屁孩、走到
屁孩終究是屁孩要就害自己就好幹嘛害別人
doc520: 屁孩、終究、幹嘛
死屁孩在盲彎敢這樣越過雙黃線超車害人害己
doc521: 害人、害己、盲彎、越過、雙黃線、超車、死屁孩
右側超車就是屁孩行為了阿
doc522: 右側、超車、屁孩
暑假這段時間還是少跑一些熱點，雖然在市區也有被屁孩擊落的可能
doc523: 市區、擊落、熱點、時間、暑假、屁孩
屁孩因該給三寶去收拾這樣世界才會好
doc524: 收拾、世界、屁孩
暑假出門不能跑山那要幹嘛當然是在路上

那種垃圾就是搞不懂抄襲和參考之間的差異
doc726: 之間、參考、差異、抄襲、垃圾
再怎麼改還是垃圾
doc727: 垃圾
不能發色情、仇恨言論或垃圾訊息那還有什麼用爛死了
doc728: 仇恨、爛死、色情、訊息、言論、垃圾
新版的跟垃圾一樣
doc729: 新版、垃圾
每次都愈改愈垃圾
doc730: 垃圾
臉書逐漸垃圾化了嗎
doc731: 垃圾化、臉書
真的垃圾我一堆建華也被鎖
doc732: 建華、真的、垃圾
你才垃圾，你後宮女人全都垃圾喔我忘了，你沒有後宮欸
doc733: 後宮、女人、垃圾
朕再三強調同人就是垃圾不應該出現在文圈之中煩請小編過審謝謝
doc734: 不應該、之中、同人、文圈、謝謝、再三、小編、垃圾
朕再三強調同人就是垃圾不應該出現在文圈之中煩請小編過審謝謝
doc735: 不應該、之中、同人、文圈、謝謝、再三、小編、垃圾
朕再三強調，你就是雷包垃圾，去刑部領大板，革職永不續用
doc736: 去刑部、大板、續用、雷包、革職、再三、垃圾
朕再三強調同人就是垃圾不應該出現在文圈之中煩請小編過審謝謝
doc737: 不應該、之中、同人、文圈、謝謝、再三、小編、垃圾
朕再三強調，原潑就是個垃圾
doc738: 原潑、再三、垃圾
可是這垃圾還是要備份個幾架在倉庫防改修
doc739: 倉庫、備份、防改修、垃圾
高雄也有另外ㄧ個光頭只不過是垃圾
doc740: 光頭、只不過、高雄、垃圾
幹這種垃圾抓出來全民給他法槌
doc741: 全民、抓出來、法槌、垃圾
國民檔垃圾
doc742: 國民檔、垃圾
這是什麼垃圾狗粉絲團
doc743: 垃圾狗、粉絲團
直接吉了百年垃圾國民黨
doc744: 國民黨、垃圾
臺北暴徒把垃圾粉專噴到關起來
doc745: 噴到、垃圾粉、暴徒、臺北
垃圾國民黨
doc746: 國民黨、垃圾
收集垃圾的人應該是病態，社區的管理人員應該好好管理了
doc747: 管理、人員、收集、病態、社區、好好、垃圾
三年前因為工作常常經過這邊每次都是堆滿垃圾
doc748: 堆滿、工作、垃圾
人類可以清除垃圾也可以製造垃圾
doc749: 清除、人類、製造、垃圾
我想或許臺一亮以外的玩家都是垃圾例如我
doc750: 玩家、垃圾
你該更正是以外的都是電子垃圾
doc751: 更正、電子、垃圾
貧果就幾乎不用調判定啊，不然你有種去拿華為垃圾